In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map() #center参数是打开地图的中心点经纬度坐标
Map

In [ ]:
start_time = '2020-01' # 开始时间
end_time = '2021-01'   # 结束时间

In [ ]:
roi = geemap.geojson_to_ee("E:/Data/GIS/JX.json")
Map.centerObject(roi)
Map.addLayer(roi, {}, "roi")

In [ ]:
def fun(img):
    return img.clip(roi.geometry()).select("sur_refl_b01", "sur_refl_b02")
def fun1(img):
    return img.addBands(img.expression(
                '(NIR-RED)/(NIR+RED)', {
                  'NIR': img.select('sur_refl_b02'),
                  'RED': img.select('sur_refl_b01')
            }).select([0], ["NDVI"])).select("NDVI")

In [ ]:
modis = ee.ImageCollection("MODIS/006/MOD09Q1")\
        .filterDate(start_time, end_time)\
        .filterBounds(roi.geometry())\
        .map(fun).map(fun1)

In [ ]:
def fun2(img, first):
    return ee.Image(first).addBands(img)

In [ ]:
modis_img = ee.Image(modis.iterate(fun2, ee.Image())).select("NDV.*?")

In [ ]:
name = modis_img.bandNames().getInfo()

In [ ]:
new_name = []
for i in range(1, 366, 8):
    new_name.append(str(i))

In [ ]:
image = modis_img.select(name, new_name)

In [ ]:
image.bandNames().getInfo()

In [ ]:
point = geemap.shp_to_ee("E:/Data/GIS/points.shp")

In [ ]:
csv = "E:/public/out_points.shp"
# geemap.extract_values_to_points(point, image, csv, geometries=True)

In [ ]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")\
        .filterDate(start_time, end_time)\
        .filterBounds(roi.geometry())

In [ ]:
landsat8.aggregate_array("system:id").getInfo()

In [ ]:
landsat8.size().getInfo()

In [ ]:
point.first().getInfo()

In [ ]:
geo = point.geometry().getInfo()

In [ ]:
geo

In [ ]:
import pandas as pd
shp = "E:/public/out_points.shp"
points = geemap.shp_to_ee(shp)

In [ ]:
df = geemap.ee_to_pandas(points)
df

In [ ]:
df1 = df.iloc[:, -2:]
df1

In [ ]:
df2 = df.iloc[:, 2:-2]
df2

In [ ]:
df3 = pd.concat([df1, df2], axis=1)
df3

In [ ]:
df3.to_csv("E:/Data/temp/Landsat_250.csv")

In [ ]:
import geemap
js_snippet = '''
function Water(img)
{
    return img.select('mNDWI').gt(img.select('EVI'))
    .or(img.select('mNDWI').gt(img.select('NDVI')))
    .and(img.select('EVI').lt(0.1));
}
'''


In [ ]:
geemap.js_snippet_to_py(js_snippet)

In [ ]:
def Water(img):
    return img.select('mNDWI').gt(img.select('EVI'))\
    .Or(img.select('mNDWI').gt(img.select('NDVI')))\
    .And(img.select('EVI').lt(0.1))